### Librerias

In [16]:
import random
import simpy

###  Bank renege

In [18]:

RANDOM_SEED = 42
NEW_CUSTOMERS = 5  # Número total de clientes
INTERVAL_CUSTOMERS = 10.0  # Se generan clientas nuevos cada 10s
MIN_PATIENCE = 1  # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
count_sim = 15 #Numero de veces que queremos que simule
times = [] #Tiempos de los clientes en la fila por cada simulación i
Xn = [] #Variables aleatorias (tiempo promedio en la fila de cada simulación i)
        #Usada para el cálculo de Sample_Mean y Sample_Variance
x_i = 0 #Almacena la media recursiva
s_i = 0 #Almacena la varianza recursiva


def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    global times
    """Customer arrives, is served and leaves."""
    arrive = env.now
    #print('%7.4f %s: Here I am' % (arrive, name))

    with counter.request() as req:
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive
        times.append(wait)
        
        if req in results:
            # We got to the counter
            
            #print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            #print('%7.4f %s: Finished' % (env.now, name))

        else:
            # We reneged
            #print('%7.4f %s: RENEGED after %6.3f' % (env.now, name, wait))
            times.append(0)

#Bloques            
print('Simulación Bank renege')
random.seed(RANDOM_SEED)            
 
for i in range(1, count_sim + 1):
    times.clear()
    print("\nSimulación", i)
    print("Se realiza con ", NEW_CUSTOMERS, " clientes")

    #Configura e inicia la simulación
    env = simpy.Environment()

    #Inicia los procesos "run" y "process"
    counter = simpy.Resource(env, capacity=1)
    env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
    env.run()
    
    xn = sum(times)/NEW_CUSTOMERS
    print("El tiempo promedio de espera de la simulación fue de: ", xn)
    
    #Calculo la media de forma recursiva
    aux = x_i
    x_i += (xn - x_i)/(i)
    print("La media de la muestra actual es:", x_i)
    
    #La varianza se calcula a partir de la simlación i+1
    if i>1:
        #Calculo la varianza de forma recursiva
        s_i = (1 - 1/(i - 1))*s_i + (i)*(x_i - aux)**2
        print("La varianza de la muestra actual es:", s_i)

Simulación Bank renege

Simulación 1
Se realiza con  5  clientes
El tiempo promedio de espera de la simulación fue de:  0.2347755330517664
La media de la muestra actual es: 0.2347755330517664

Simulación 2
Se realiza con  5  clientes
El tiempo promedio de espera de la simulación fue de:  0.23868655073338516
La media de la muestra actual es: 0.23673104189257577
La varianza de la muestra actual es: 7.648029652967143e-06

Simulación 3
Se realiza con  5  clientes
El tiempo promedio de espera de la simulación fue de:  1.4878093828640373
La media de la muestra actual es: 0.6537571555497296
La varianza de la muestra actual es: 0.5217361624307945

Simulación 4
Se realiza con  5  clientes
El tiempo promedio de espera de la simulación fue de:  0.6548635156663984
La media de la muestra actual es: 0.6540337455788968
La varianza de la muestra actual es: 0.3478244142953733

Simulación 5
Se realiza con  5  clientes
El tiempo promedio de espera de la simulación fue de:  0.5266302116167871
La media de 

### Media

In [6]:
def sample_mean(list_):
    sol = sum(list_)/len(list_)
    return sol

### Varianza muestral

In [7]:
def sample_variance(list_, X):
    S = 0
    for i in range(len(list_)):
        S = S + (list_[i] - X)**2
    sol = S/(len(list_) - 1)
    return sol

In [21]:
examples_k = [100, 1000, 10000]

random.seed(RANDOM_SEED)

for k in examples_k:
    for i in range(k):
        times.clear()

        #Configura e inicia la simulación
        env = simpy.Environment()

        #Inicia los procesos "run" y "process"
        counter = simpy.Resource(env, capacity=1)
        env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
        env.run()

        xn = sum(times)/NEW_CUSTOMERS
        Xn.append(xn)
    
    X = sample_mean(Xn)
    print("\n")
    print("Media para k=",k," es: ", X)
    print("Varianza muestral para k=",k," es: ", sample_variance(Xn, X))
    Xn = []



Media para k= 100  es:  0.8245370179904764
Varianza muestral para k= 100  es:  0.23084066897002462


Media para k= 1000  es:  0.843245319074124
Varianza muestral para k= 1000  es:  0.1903044898097185


Media para k= 10000  es:  0.8515551323732156
Varianza muestral para k= 10000  es:  0.20193676148951067
